<a href="https://colab.research.google.com/github/KizMan-23/sentiment_analysis/blob/main/spotify_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files

upload = files.upload()

Saving Spotify.csv to Spotify.csv


In [5]:
import pandas as pd
import re

In [6]:
spotify = pd.read_csv('Spotify.csv')

spotify.head()

,Review,label
0,"Great music service, the audio is high quality...",POSITIVE
1,Please ignore previous negative rating. This a...,POSITIVE
2,"This pop-up ""Get the best Spotify experience o...",NEGATIVE
3,Really buggy and terrible to use as of recently,NEGATIVE
4,Dear Spotify why do I get songs that I didn't ...,NEGATIVE


In [7]:
label_map = {
    "POSITIVE": 1,
    "NEGATIVE": 0
}

spotify['label'] = spotify['label'].map(label_map)

In [8]:
spotify.head()

,Review,label
0,"Great music service, the audio is high quality...",1
1,Please ignore previous negative rating. This a...,1
2,"This pop-up ""Get the best Spotify experience o...",0
3,Really buggy and terrible to use as of recently,0
4,Dear Spotify why do I get songs that I didn't ...,0


In [9]:

spotify.shape

(52702, 2)

In [10]:
spotify.isnull().sum()

,0
Review,16
label,0


In [11]:
empty = spotify[spotify['Review'].isnull()]

spotify = spotify.dropna()
spotify.reset_index(inplace=True,drop=True)

In [12]:
spotify.shape

(52686, 2)

In [13]:
#distribution of sentiments
spotify['label'].value_counts()

#Negative sentiment is relatively higher

,count
label,
0,29423
1,23263


TEXT PROCESSING

In [14]:
def process_text(text):
    pattern = r'\b[a-zA-Z]+\b'
    new_text = " ".join(re.findall(pattern,text))
    new_text = str(new_text).lower()
    return new_text

In [15]:
spotify['Review'] = spotify['Review'].apply(process_text)

In [16]:
spotify['Review'].head(10)

,Review
0,great music service the audio is high quality ...
1,please ignore previous negative rating this ap...
2,this pop up get the best spotify experience on...
3,really buggy and terrible to use as of recently
4,dear spotify why do i get songs that i didn t ...
5,the player controls sometimes disappear for no...
6,i love the selection and the lyrics are provid...
7,still extremely slow when changing storage to ...
8,it s a great app and the best music app i have...
9,i m deleting this app for the following reason...


In [32]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.ensemble import HistGradientBoostingClassifier
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

In [33]:
#My session(RAM) continues to crash due to data overload
#I have to shrink my data to 20,000

pos_df = spotify[spotify['label']==1].sample(10000,random_state=42)
neg_df = spotify[spotify['label']==0].sample(10000,random_state=42)

spotify = pd.concat([pos_df,neg_df]).sample(frac=1, random_state=1) #shuffle the mixture
spotify.reset_index(drop=True, inplace=True)


In [19]:
nlp = spacy.load('en_core_web_sm')

In [20]:
doc = [nlp(i) for i in spotify['Review']]

In [26]:
vectorizer = TfidfVectorizer()

x_tf = vectorizer.fit_transform(spotify['Review']).toarray()
x_nlp = np.array([word.vector for review in doc for word in review])
x_nlp = x_nlp.reshape(-1,96)
y = spotify['label'].values

In [27]:
x_nlp

array([[-0.6514269 , -0.3890745 ,  0.39770278, ...,  1.4447479 ,
        -0.6219163 ,  0.48288316],
       [-1.2481502 , -0.5929848 ,  0.27870256, ...,  1.0982423 ,
         1.322463  ,  0.5476479 ],
       [ 0.09505588, -0.9051567 , -0.35835332, ...,  0.6074271 ,
         0.17391802, -0.3984886 ],
       ...,
       [ 0.00948605, -0.6094815 ,  0.44357607, ...,  0.40324795,
        -0.6650138 , -0.1661577 ],
       [ 1.0964177 , -0.5848948 , -0.5179785 , ...,  0.7198286 ,
        -0.4868574 , -1.247194  ],
       [-1.1095568 , -0.63560873, -0.8702686 , ...,  0.22466841,
         0.5927578 ,  1.3591492 ]], dtype=float32)

In [28]:
x_train,x_test,y_train,y_test = train_test_split(x_tf,y, test_size=0.3, random_state=42)

MULTINOMIAL NAIVE_BAYES

In [30]:
mul = MultinomialNB()
mul.fit(x_train,y_train)

MultinomialNB()

In [31]:
mul_preds = mul.predict(x_test)

In [52]:
#Classification Metric

print(accuracy_score(y_test,mul_preds))
print(classification_report(y_test,mul_preds))

0.8611666666666666
              precision    recall  f1-score   support

           0       0.84      0.90      0.87      3018
           1       0.89      0.82      0.85      2982

    accuracy                           0.86      6000
   macro avg       0.86      0.86      0.86      6000
weighted avg       0.86      0.86      0.86      6000



In [36]:
pd.crosstab(y_test,mul_preds, rownames=['Actual'], colnames=['Predicted'])

Predicted,0,1
Actual,,
0,2722,296
1,537,2445


LINEARSVC

In [37]:
svc_clf = LinearSVC(random_state=42)
svc_clf.fit(x_train,y_train)
svc_preds = svc_clf.predict(x_test)

In [51]:
#LinearSVC Metric

print(accuracy_score(y_test,svc_preds))
print(classification_report(y_test,svc_preds))

0.8791666666666667
              precision    recall  f1-score   support

           0       0.88      0.89      0.88      3018
           1       0.88      0.87      0.88      2982

    accuracy                           0.88      6000
   macro avg       0.88      0.88      0.88      6000
weighted avg       0.88      0.88      0.88      6000



In [39]:
pd.crosstab(y_test,svc_preds)

col_0,0,1
row_0,,
0,2671,347
1,378,2604


*HISTGRADIENTBOOSTING* *CLASSIFICATION*

In [45]:
hst_clf = HistGradientBoostingClassifier(random_state=42, max_iter=200)

hst_clf.fit(x_train,y_train)


In [46]:
hst_preds = hst_clf.predict(x_test)

In [47]:
pd.crosstab(y_test,hst_preds)

col_0,0,1
row_0,,
0,2666,352
1,408,2574


In [50]:
print(accuracy_score(y_test,hst_preds))
print(classification_report(y_test,hst_preds))


0.8733333333333333
              precision    recall  f1-score   support

           0       0.87      0.88      0.88      3018
           1       0.88      0.86      0.87      2982

    accuracy                           0.87      6000
   macro avg       0.87      0.87      0.87      6000
weighted avg       0.87      0.87      0.87      6000

